In [1]:
sys.path.insert(0, '/home/matheus/Documentos/house-recommendation/src')
from joblib import load
from model_inference import ModelInference
from model_training import ModelTraining
from experiments import Experiments
from data_source import DataSource

In [2]:
ModelInference().predict()

Loading model
Loading data
Preprocessing Data
Creating dataframe for data manipulation
Droping columns with missing values
Dropping column with id
Creating list with numeric features
Creating list with categorical features
removing target
['bairro']
feature encoder
feature normalization and encoding
bairro       0
area         0
quartos      0
garagem      0
banheiros    0
preco        0
dtype: int64
Predicting
Evaluating model in test data
{'roc_auc_score': 0.9657689305230288, 'average_precision_score': 0.8834351299095599}
Saving file


array([1.44702025e-03, 5.28594170e-03, 9.99393933e-01, 2.80946047e-03,
       3.70658940e-05, 2.95885975e-03, 7.03531724e-05, 6.01376673e-06,
       1.32812026e-05, 9.56428924e-01, 2.80411033e-05, 5.90012601e-01,
       5.13792380e-02, 9.72124875e-01, 9.29504622e-01, 2.40053384e-05,
       1.24443726e-01, 7.41986489e-05, 2.12336340e-02, 1.05348740e-01,
       2.77306194e-05, 1.09432633e-01, 1.12746180e-03, 5.59698091e-01,
       2.52885226e-01, 1.74531362e-05, 3.35069867e-04, 2.25051089e-05,
       5.15051362e-06, 5.71160836e-04, 4.77154824e-02, 2.62883860e-01,
       2.22042653e-03, 1.33907965e-01, 9.99262321e-01, 5.27872495e-02,
       4.05830724e-02, 3.05286964e-04, 1.43298183e-05, 1.47187257e-05,
       6.48480309e-05, 1.11962064e-05, 3.51705365e-04, 1.39501147e-02,
       9.83915065e-01, 1.64972983e-05, 5.63430190e-02, 6.47848718e-05,
       1.08093992e-02, 2.30795519e-05, 2.95836068e-04, 5.46962378e-02,
       5.63106433e-05, 4.80024909e-03, 9.94053862e-02, 2.64926520e-04,
      